# Cuaderno 23: Modelo de flujos para la asignación de vehículos

*Este cuaderno contiene material tomado del Trabajo de Integración Curricular de Mauricio Yépez, en la carrera de Ingeniería Matemática en la Escuela Politécnica Nacional (Quito, 2022).*

En el problema de asignación de vehículos (*Vehicle Scheduling Problem, VSP*) se tienen dados como entrada el conjunto de viajes que deben cumplirse a lo largo del día en un (sub)sistema de transportación pública, y la flota de vehículos (buses, trolebuses, tranvías, trenes, etc) disponible para cubrir estos viajes. Cada viaje tiene asociados, entre otra información:
* una estación de inicio,
* una hora de inicio,
* una estación de finalización, y,
* una hora de finalización.

Un vehículo y un viaje se dicen *compatibles* si el vehículo tiene las características técnicas necesarias para cubrir el viaje. 

Dos viajes se dicen *compatibles* si es posible para un mismo vehículo cubrir el un viaje a continuación del otro. Notar que la compatibilidad es una relación antisimétrica: si un vehículo puede cubrir un viaje $b$ inmediatamente después de un viaje $a$, esto significa que la hora de finalización de $a$ es menor o igual a la hora de inicio de $b$, por lo que es imposible que un vehículo cubra $a$ inmediatamente después de $b$.

Además de las horas de inicio y finalización, pueden existir otros criterios para definir cuáles viajes son compatibles entre sí. Estos criterios son muy variados y dependen de la aplicación en particular. Por ejemplo, puede requerirse que la estación de finalización del primer viaje coincida con la estación de inicio del segundo, o que estas estaciones estén cercanas entre sí, para limitar la distancia total de los "viajes vacíos" de la flota. 

La tarea en el problema de asignación de vehículos consiste en construir rutas de servicio para los vehículos de la flota. Cada ruta de servicio consiste de una sucesión de viajes compatibles a ser cubiertos por una misma unidad (la que, desde luego, deberá ser compatible con todos los viajes de la ruta). Cada viaje programado en el (sub)sistema debe estar cubierto exactamente por una ruta de servicio, y ninguna unidad de la flota puede tener asignadas más de una ruta de servicio. La función objetivo a minimizar mide generalmente algún componente (o varias componentes) de los costos de operación del sistema:
* número de unidades (vehículos) requeridos;
* distancia total en desplazamientos vacíos entre viajes;
* tiempo total en desplazamientos vacíos entre viajes;
* tiempos de espera de las unidades entre viajes;
entre otros.

Consideraremos a continuación un ejemplo donde se tiene una *flota de vehículos homogénea*, es decir, todos los vehículos son compatibles con todos los viajes. Para formular este problema, construimos una red $D=(V,A)$ con
* Un conjunto de nodos $V:= N \cup \{s,t\}$ que incluye: 
    - un conjunto $N$ de nodos que representan los viajes a ser cubiertos; 
    - un nodo fuente $s$ y un nodo sumidero $t$.
* Un conjunto de arcos $A$ conformado por
    - arcos $(i,j)$, con $i,j \in N$, si y solamente si el viaje $i \in N$ es compatible con el viaje $j \in N$, es decir, si $j$ puede ser cubierto inmediatamente después de $i$ por alguna unidad de la flota;
    - arcos desde $s$ hacia cada viaje $i\in N$; y,
    - arcos desde cada viaje $i\in N$ hacia $t$.
    
Sobre los arcos de la red definimos un vector de costos $c \in \mathbb{R}^A$ de la siguiente manera: cada arco de la forma $(s,i)$ con $i \in N$, tiene asociado el costo fijo de utilizar una unidad de transporte, cada arco de la forma $(i,j)$, con $i,j \in N$ tiene asociado el costo relativo al tiempo de espera y al desplazamiento vacío de una unidad cuando cubre el viaje $j$ inmediatamente después del viaje $i$; y cada arco de la forma $(i,t)$, con $i \in N$, tiene costo cero.

De esta manera, el problema de asignación de vehículos consiste en encontrar un conjunto de caminos o rutas desde $s$ hasta $t$, de tal manera que cada viaje $i\in N$ sea cubierto exactamente por una ruta, y que el costo total de los arcos seleccionados dentro de las rutas sea mínimo. Empleando variables binarias de selección de arcos, este problema puede formularse como el siguiente programa lineal entero:    
    
\begin{align}
\min&\sum_{(i,j)\in A}c_{ij}x_{ij}\\
s.t.&\sum_{(j,i)\in A}x_{ji}-\sum_{(i,j)\in A}x_{ij}=0, &\forall i\in N,\\
&\sum_{(i,j)\in A}x_{ij}=1, &\forall i\in N,\\
&x_{ij}\in\{0,1\}, &\forall (i,j)\in A.
\end{align}

Vamos a implementar este modelo empleando el API Python de Gurobi sobre la instancia indicada a continuación:

La red de transporte $PTN=(\mathcal{V}, \mathcal{A})$ corresponde al digrafo indicado en la siguiente figura:

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import matplotlib.pyplot as plt

VV=["v"+str(i) for i in range(1,7)]
AA=[("v1","v2"),("v2","v3"),("v3","v4"),("v5","v2"),("v2","v3"),("v3","v6")]

D = nx.DiGraph()
D.add_nodes_from(VV)
node_labels= {i : str(i) for i in VV}
D.add_edges_from(AA)
plt.figure(figsize=(10,5))
h=1
pos = {"v1" : (0,h), "v2" : (2,h/2), "v3" : (5,h/2), "v4" : (7,h), "v5" : (0,0), "v6":(7,0)}
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=1500)

Sobre está red operan dos líneas: la primera realiza viajes por las estaciones $(v_1,v_2,v_3,v_4)$, mientras que la segunda recorre las estaciones $(v_5,v_2,v_3,v_6)$. Se han calendarizado un total de ocho viajes de estas dos líneas a lo largo de cuatro períodos, conforme se indica en la siguiente figura:

In [ ]:
fs=12
P=list(range(1,5))
VV=["v"+str(i)+"_"+str(j) for i in [1,4,5,6] for j in P]
AA=[("v1_"+str(j),"v4_"+str(j)) for j in P]+[("v5_"+str(j),"v6_"+str(j)) for j in P]
D = nx.DiGraph()
D.add_nodes_from(VV)
node_labels= {i : " " for i in VV}
D.add_edges_from(AA)
plt.figure(figsize=(10,10))
pos = dict(dict({"v1_"+str(j): (4,39-3*j) for j in P},**{"v4_"+str(j): (6,37-3*j) for j in P}),
          **dict({"v5_"+str(j):(7.5,40-3*j) for j in P},**{"v6_"+str(j): (9.5,36-3*j) for j in P}))
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=500)
for j in P:
    plt.plot([2, 10], [40-3*j,40-3*j],'k--',lw=1)
    plt.text(1.65, 39.9-3*j, str(j)+":00", fontsize = fs)    
    plt.text(1.65, 38.5-3*j, "Periodo "+str(j), fontsize = fs)
plt.plot([2, 10], [40-3*5,40-3*5],'k--',lw=1)
plt.text(4, 37.5, "v1", fontsize = fs)
plt.text(6, 37.5, "v4", fontsize = fs)
plt.text(7.5, 37.5, "v5", fontsize = fs)
plt.text(9.5, 37.5, "v6", fontsize = fs)
plt.show()

A partir de la información de la calendarización (periódica) de estos viajes, construimos el conjunto $N$ de viajes a cubrir. Almacenaremos los datos correspondientes en un diccionario `trips` cuyas claves corresponden a los viajes, y donde el valor asociado a cada clave es un subdiccionario que contiene información de la hora de salida `s`, la hora de llegada `e`, la estación de salida `SL` y la estación de llegada `EL` de cada viaje.

In [ ]:
#Lineas en la PTN
l=list(range(1,3))
#Periodos de planificacion de viajes
P=list(range(1,5))
#tiempos de inicio de cada viaje en cada periodo
pfirst = {1: 20,2:0}
#duracion de cada viaje en cada periodo
dl = {1:40,2:80}
#Horizonte de planificación
T = 60
#estaciones en donde la linea i inicia, i=1,2
SL = {1:"v1",2:"v5"}
#estaciones en donde la linea i termina, i=1,2
EL = {1:"v4",2:"v6"}
#viajes a cubrirse en la red de transporte público
trips = {"l"+str(i)+"_"+str(p):{"s":p*T+pfirst[i],"e":p*T+pfirst[i]+dl[i],
                                "SL":SL[i],"EL":EL[i]} for i in l for p in P}

for k,val in trips.items():
    print(k,val)

Dos viajes serán compatibles entre sí, si el tiempo de desplazamiento de un vehículo desde la estación final del primer viaje hasta la estación inicial del primer viaje es menor o igual a la diferencia de tiempos entre la finalización del primer viaje y el inicio del segundo viaje. 

Para poder determinar el conjunto de viajes compatibles necesitamos, por lo tanto, información acerca de los tiempos de desplazamiento entre las estaciones finales y las estaciones iniciales de los viajes. Asumiremos que esta información viene dada en el siguiente diccionario `t`, donde `t[vi,vj]`es el tiempo para desplazarse desde la estación `vi` hasta la estación `vj`:  

In [ ]:
t = {('v4','v1') : 19, ('v4','v5') : 59, 
     ('v6','v1') : 59, ('v6','v5') : 39}

Con los datos anteriores, definimos la red $D=(V,A)$:

In [ ]:
# Conjunto de viajes a realizarse:
N = gp.tuplelist(trips.keys())

# Vértices de la red
V = N + ["s", "t"]

# por cada par de viajes i,j, se añade un arco (i,j) solo si los viajes son compatibles,
# i.e, si existe tiempo suficiente para realizar un viaje luego de terminar otro.
A = [(i,j) for i in N for j in N if (trips[j]['s']-trips[i]['e']>=
                                     t[trips[i]["EL"],trips[j]["SL"]] and i!=j)]

# se conecta el nodo fuente con cada viaje y cada viaje con el nodo sumidero
A = A + [("s",j) for j in N] + [(i,"t") for i in N]


Para este ejemplo, asumiremos que nos interesa minimizar el número de vehículos a utilizar. Por lo tanto, los costos sobre los arcos serán iguales a 1, para los arcos que salen desde el nodo fuente $s$, y a 0 en los demás casos. Esto garantiza que cada ruta de $s$ a $t$ tenga un costo igual a 1:

In [ ]:
#para contar el número de rutas necesarias se coloca como 1 el costo de los viajes vacios desde
# el depósito hacia los viajes y como 0 los demas.
c= {(i,j):1 if i=="s" else 0 for (i,j) in A}


Con los datos anteriores, construimos el modelo de programación lineal entera:

In [ ]:
# crear objeto modelo
mf = gp.Model("VSPMF0")

# crear variables de selección de arcos
x = mf.addVars(A, vtype=GRB.BINARY, name="x")

# crear función objetivo
mf.setObjective(x.prod(c), GRB.MINIMIZE)

# añadir restricciones de conservación de flujo
mf.addConstrs((x.sum('*',j) - x.sum(j,'*') == 0 for j in N), name="flow")

# añadir restricciones de grado
mf.addConstrs((x.sum('*',j) == 1 for j in N), name="grado")


Podemos exportar este modelo a un archivo en formato `lp`:

In [ ]:
mf.write("VSP0.lp")

Finalmente, resolvemos el modelo:

In [ ]:
mf.optimize()

Una vez resuelto el modelo, podemos recuperar de los valores de las variables en la solución las rutas de servicio y la cantidad de vehículos a utilizar:

In [ ]:
# Construir lista con arcos seleccionados por el modelo
arcos_rutas = gp.tuplelist([(i,j) for (i,j) in A if x[i,j].x>=0.9])
# Construir lista de lista de rutas
rutas = []
# Inspeccionar arcos que salen de s
for (i,j) in arcos_rutas.select("s", '*'):
    T = [i, j]
    # Avanzar nodo a nodo en la ruta, hasta llegar a t
    while j!='t':
        a = arcos_rutas.select(j,'*')[0]
        j = a[1]
        T.append(j)
    rutas.append(T)

for T in rutas:
    print(T)
print('Número de vehículos: {}'.format(len(rutas)))

También es posible graficar las rutas:

In [ ]:
if mf.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = mf.getAttr('x', x)
P=list(range(1,5))
VV=["v"+str(i)+"_"+str(j) for i in [1,4,5,6] for j in P]
VS=[("v1_"+str(j),"v4_"+str(j)) for j in P]+[("v5_"+str(j),"v6_"+str(j)) for j in P]+[(trips[i]["EL"]+i[2:],trips[j]["SL"]+j[2:]) for (i,j) in A if (vx[i,j]>0.9 and i!="s" and j!="t")]
D = nx.DiGraph()
D.add_nodes_from(VV)
node_labels= {i : " " for i in VV}
D.add_edges_from(VS)
plt.figure(figsize=(10,10))
pos = dict(dict({"v1_"+str(j): (4,39-3*j) for j in P},**{"v4_"+str(j): (6,37-3*j) for j in P}),
          **dict({"v5_"+str(j):(7.5,40-3*j) for j in P},**{"v6_"+str(j): (9.5,36-3*j) for j in P}))
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=500)
for j in P:
    plt.plot([2, 10], [40-3*j,40-3*j],'k--',lw=1)
    plt.text(1.65, 39.9-3*j, str(j)+":00", fontsize = fs)    
    plt.text(1.65, 38.5-3*j, "Periodo "+str(j), fontsize = fs)
plt.plot([2, 10], [40-3*5,40-3*5],'k--',lw=1)
plt.text(4, 37.5, "v1", fontsize = fs)
plt.text(6, 37.5, "v4", fontsize = fs)
plt.text(7.5, 37.5, "v5", fontsize = fs)
plt.text(9.5, 37.5, "v6", fontsize = fs)
plt.show()